Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Install and Import Libraries
Let's install PySpark:

In [70]:
import pyspark

## Build Spark Session

In [71]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

In [72]:
from pyspark.sql.functions import *

## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [73]:
schema="PassengerId INT,Survived INT,Pclass INT, Name string, Sex string,Age INT,SibSp INT,Parch INT,Ticket string,Fare INT,Cabin string,Embarked string"

In [74]:
Train=spark.read.csv('train.csv',schema=schema,header=True)
Test=spark.read.csv('test.csv',schema=schema,header=True)

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [75]:
print(Train.dtypes)

[('PassengerId', 'int'), ('Survived', 'int'), ('Pclass', 'int'), ('Name', 'string'), ('Sex', 'string'), ('Age', 'int'), ('SibSp', 'int'), ('Parch', 'int'), ('Ticket', 'string'), ('Fare', 'int'), ('Cabin', 'string'), ('Embarked', 'string')]


**Show 5 rows.**

In [76]:
print(Train.show(5))

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|null| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|null|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|null| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|null| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|null| null|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-----+--------+
o

**Display schema for the dataset:**

In [77]:
Train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: integer (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [78]:
Train.summary().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               689|               891|                891|               891|              161|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null|29.847605224963715|0.5230078563411896|0.38159371492704824|260318.54916792738|33.472049

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [79]:
Train.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [80]:
#Train.createOrReplaceTempView("Survived_table")

In [81]:
#survived_count=spark.sql("select Survived,count(Survived) as count from Survived_table group by Survived")

In [82]:
survived_count=Train.groupBy('Survived').count()

**Display your result:**

In [83]:
survived_count.show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [84]:
count=Train.select('Survived').count()

**Can you display your answer in ratio form?(Hint: Use UDF.)**






In [85]:
@udf(returnType=StringType()) 
def ratio_survived(num):
    return num/count
spark.udf.register("ratio_survived", lambda str: ratio_survived(str) if not str is None else "UNKNOWN" , StringType())
survived_count.withColumn("Ratio", ratio_survived(col("count"))).show(truncate=False)

+--------+-----+------------------+
|Survived|count|Ratio             |
+--------+-----+------------------+
|1       |342  |0.3838383838383838|
|0       |549  |0.6161616161616161|
+--------+-----+------------------+



**Can you get the number of males and females?**


In [86]:
Train.groupBy('Sex').count().show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column.)

In [87]:
#1
Train.groupBy('Sex').agg(avg(col('Survived'))).show()

+------+-------------------+
|   Sex|      avg(Survived)|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+



In [88]:
#2
Train.groupBy('Sex').agg(sum(col('Survived'))).show()

+------+-------------+
|   Sex|sum(Survived)|
+------+-------------+
|female|          233|
|  male|          109|
+------+-------------+



**Create temporary view PySpark:**

In [89]:
Train.createOrReplaceTempView("Survived_table")

**How many people survived, and how many didn't survive? By SQL:**

In [90]:
survived_count_2=spark.sql("select Survived,count(Survived) as count from Survived_table group by Survived")

In [91]:
survived_count_2.show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column.)

**Can you do this via SQL?**

In [92]:
spark.sql('select Sex,count(Survived) from Survived_table group by Sex').show()

+------+---------------+
|   Sex|count(Survived)|
+------+---------------+
|female|            314|
|  male|            577|
+------+---------------+



**Display a ratio for p-class:**


In [93]:
spark.sql('select Pclass,count(*)/sum(count(*)) over() as ratio from Survived_table as ratio  group by Pclass').show()

+------+-------------------+
|Pclass|              ratio|
+------+-------------------+
|     1|0.24242424242424243|
|     3| 0.5510662177328844|
|     2|0.20650953984287318|
+------+-------------------+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [94]:
combined=Train.union(Test)

**Display count:**

In [95]:
combined.count()

1329

**Temporary view PySpark:**

In [96]:
combined.createOrReplaceTempView("dataset_table")

**Can you define the number of null values in each column?**


In [97]:
from pyspark.sql.functions import when, count, col
combined.select([count(when(isnull(c), c)).alias(c) for c in combined.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|301|    0|    0|     0|1090| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [98]:
combined.select([count(when(isnull(c), c)).alias(c) for c in combined.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|301|    0|    0|     0|1090| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



## Preprocessing 

**Can you show me the name column from your temporary table?**

In [99]:
spark.sql('select Name from dataset_table').show()

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



**Run this code:**

In [100]:
import pyspark.sql.functions as F

In [101]:
combined = combined.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
combined.createOrReplaceTempView('combined')

**Display the title and count "Title" column:**

In [102]:
title_count=combined.groupBy('Title').count()

In [103]:
title_count.show()

+--------+-----+
|   Title|count|
+--------+-----+
|     Don|    1|
|    Miss|  257|
|Countess|    2|
|     Col|    4|
|     Rev|    9|
|    Lady|    2|
|  Master|   56|
|     Mme|    1|
|    Capt|    2|
|      Mr|  786|
|      Dr|   11|
|     Mrs|  186|
|     Sir|    2|
|Jonkheer|    2|
|    Mlle|    4|
|   Major|    3|
|      Ms|    1|
+--------+-----+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [104]:
titles_map={row[0]:row[0] if row[1]>11 else 'rare' for row in title_count.collect()}

In [105]:
titles_map

{'Don': 'rare',
 'Miss': 'Miss',
 'Countess': 'rare',
 'Col': 'rare',
 'Rev': 'rare',
 'Lady': 'rare',
 'Master': 'Master',
 'Mme': 'rare',
 'Capt': 'rare',
 'Mr': 'Mr',
 'Dr': 'rare',
 'Mrs': 'Mrs',
 'Sir': 'rare',
 'Jonkheer': 'rare',
 'Mlle': 'rare',
 'Major': 'rare',
 'Ms': 'rare'}

**Run the function:**

In [106]:
@udf(returnType=StringType()) 
def impute_title(title):
    return titles_map[title]

**Apply the function on "Title" column using UDF:**

In [107]:
combined=combined.withColumn("Title", impute_title(col("Title")))

**Display "Title" from table and group by "Title" column:**

In [108]:
combined.groupBy('Title').count().show()

+------+-----+
| Title|count|
+------+-----+
|  rare|   44|
|  Miss|  257|
|Master|   56|
|    Mr|  786|
|   Mrs|  186|
+------+-----+



## **Preprocessing Age**

**Based on the age mean, you will fill in the missing age values:**

In [109]:
mean_age=combined.agg(avg('Age')).collect()[0][0]

In [110]:
mean_age

30.32295719844358

**Fill missing age with age mean:**

In [111]:
combined=combined.na.fill(mean_age,subset=['Age'])

## **Preprocessing Embarked**

**Select Embarked, count them, order by count Desc, and save in grouped_Embarked variable:**




In [112]:
grouped_Embarked=combined.groupBy('Embarked').count().orderBy('count',ascending=False)

**Show groupped_Embarked:**

In [113]:
grouped_Embarked.show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  962|
|       C|  253|
|       Q|  111|
|    null|    3|
+--------+-----+



**Get the groupped_Embarked:** 

In [114]:
value_embarked=grouped_Embarked.first()['Embarked']

In [115]:
value_embarked

'S'

In [116]:
modified_combined=combined

**Fill missing values with grouped_Embarked:**

In [117]:
modified_combined=modified_combined.na.fill({'Embarked':value_embarked})

In [118]:
modified_combined.select([count(when(isnull(c), c)).alias(c) for c in modified_combined.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Title|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|1090| 1021|       0|    0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+



## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



In [119]:
@udf(returnType=StringType()) 
def cabin_split(str):
    if str is None:
        return None
    else:
        return str[0]

modified_combined=modified_combined.withColumn("Cabin", cabin_split(col("Cabin")))

**Show the result:**

In [120]:
modified_combined.select('Cabin').show()

+-----+
|Cabin|
+-----+
| null|
|    C|
| null|
|    C|
| null|
| null|
|    E|
| null|
| null|
| null|
|    G|
|    C|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
+-----+
only showing top 20 rows



**Create the temporary view:**

In [121]:
modified_combined.createOrReplaceTempView('temp_view')

**Select "Cabin" column, count Cabin column, Group by "Cabin" column, Order By count DESC**  

In [122]:
modified_combined.groupBy('Cabin').count().orderBy('count',ascending=False).show()

+-----+-----+
|Cabin|count|
+-----+-----+
| null| 1021|
|    C|   82|
|    B|   77|
|    D|   52|
|    E|   51|
|    A|   23|
|    F|   18|
|    G|    4|
|    T|    1|
+-----+-----+



**Fill missing values with "U":**

In [123]:
modified_combined=modified_combined.na.fill({'Cabin':'U'})

In [124]:
modified_combined.select([count(when(isnull(c), c)).alias(c) for c in modified_combined.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Title|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|1090|    0|       0|    0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+



**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

In [143]:
from pyspark.ml.feature import StringIndexer

In [173]:
categoricalCols = [field for (field, dataType) in modified_combined.dtypes
                   if ((dataType=='string')& (field!='Name')&(field!='Ticket'))]

In [174]:
indexOutputCols = [x + "_Index" for x in categoricalCols]
indexOutputCols

['Sex_Index', 'Cabin_Index', 'Embarked_Index', 'Title_Index']

In [175]:
stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=indexOutputCols,
                             handleInvalid='skip')

In [176]:
numericCols = [field for (field,dataType) in modified_combined.dtypes
              if ((dataType=='int')& (field!='Survived')&(field!='PassengerId')&(field!='Fare'))]

In [177]:
numericCols

['Pclass', 'Age', 'SibSp', 'Parch']

In [178]:
assemblerInputs = indexOutputCols + numericCols

In [179]:
assemblerInputs

['Sex_Index',
 'Cabin_Index',
 'Embarked_Index',
 'Title_Index',
 'Pclass',
 'Age',
 'SibSp',
 'Parch']

In [180]:
from pyspark.ml.feature import VectorAssembler

In [181]:
vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

In [182]:
vecAssembler

VectorAssembler_9cf74d229570

____________________________________________

**Use Pipline to fit and transform:**

In [235]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

In [236]:
clf = RandomForestClassifier(labelCol='Survived',featuresCol='features')
pipeline =Pipeline(stages = [stringIndexer,vecAssembler,clf])

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None) A feature transformer that merges multiple columns into a vector column.**



**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [237]:
trainDF, testDF=modified_combined.randomSplit([0.8,0.2],seed=42)

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [238]:
pipelineModel = pipeline.fit(trainDF)

In [239]:
predDF = pipelineModel.transform(testDF)

In [240]:
predDF.select('prediction','Survived','features').show()

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       1.0|       1|(8,[0,3,4,5],[1.0...|
|       0.0|       0|(8,[1,4,5],[4.0,1...|
|       0.0|       1|[1.0,0.0,0.0,2.0,...|
|       0.0|       0|(8,[4,5,6,7],[3.0...|
|       1.0|       1|[1.0,0.0,1.0,2.0,...|
|       0.0|       1|(8,[1,4,5],[5.0,1...|
|       0.0|       0|(8,[4,5],[3.0,30.0])|
|       0.0|       0|(8,[4,5,6],[1.0,4...|
|       0.0|       0|(8,[4,5],[3.0,30.0])|
|       0.0|       0|(8,[2,4,5,6],[2.0...|
|       1.0|       1|[1.0,0.0,2.0,1.0,...|
|       1.0|       0|[1.0,0.0,0.0,2.0,...|
|       0.0|       0|(8,[4,5],[3.0,21.0])|
|       0.0|       1|(8,[1,4,5],[1.0,1...|
|       0.0|       0|(8,[1,4,5,6],[1.0...|
|       0.0|       0|(8,[4,5,6],[3.0,2...|
|       0.0|       0|(8,[4,5],[3.0,30.0])|
|       1.0|       0|(8,[0,3,4,5],[1.0...|
|       0.0|       0|(8,[4,5],[3.0,22.0])|
|       0.0|       0|(8,[2,4,5],[2.0,3...|
+----------

In [241]:
from pyspark.sql.types import ArrayType,DoubleType

**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [242]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [243]:
accuracy = MulticlassClassificationEvaluator(labelCol='Survived',
                                            predictionCol='prediction',
                                            metricName='accuracy').evaluate(predDF)

In [245]:
print('Accuracy:',accuracy)

Accuracy: 0.8425531914893617
